In [ ]:
import numpy as np
import pandas as pd
from pyathena import connect

In [ ]:
conn = connect(
    s3_staging_dir='s3://<user-defined>/tracking-the-sun', ##user defined staging directory
    region_name='us-west-2',
    work_group='<USER SPECIFIC WORKGROUP>'  ##specify workgroup if exists
)

df = pd.read_sql("SELECT * FROM oedi_tracking_the_sun_2019 limit 8;", conn)